Selección de la base de datos

Se utilizará la base de datos https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset, la cual contiene datos acerca de estudiantes de escuela secundaria, sus actividades extracurriculares, las horas de estudio, etc. 


El objetivo de la red a realizar es, a partir de los datos provistos, poder predecir el rendimiento académico de los alumnos, y determinar qué características influyen en dichos resultados.

*Tipos de datos:
-ID del estudiante
-Edad
-Género
-Etnia
-Educación máxima alcanzada por los padres
-Horas semanales de estudio
-Ausencias
-Si reciben tutoría especial
-El nivel de apoyo de los padres
-Si realiza actividades extracurriculares
-Si

ANOTACIONES

-El tipo de carrera puede estar relacionado con la nota final (carreras mas dificiles pueden hacer que las notas sean bajas)
-Las variables pueden pasarse a valores entre 0 y 1 para trabajarlas más fácil